---This is the code for predicting aesthetic scores or histograms using the method in the paper:
---Jin, Bin, Maria V. Ortiz Segovia, and Sabine Süsstrunk. "Image aesthetic predictors based on weighted CNNs." Image Processing (ICIP), 2016 IEEE International Conference on. IEEE, 2016.



In [1]:
from __future__ import division
import sys
sys.path.insert(0, '../MyPackages/')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2, os, random, time
from tqdm import tqdm
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2


Using Theano backend.
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '25372' (I am process '39153')
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: TITAN X (Pascal) (CNMeM is disabled, cuDNN 5004)


In [ ]:
def AestheticNet(outputflag='score',weight_decay=0.0005, weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2))) # The receptive filed is 7*7 here

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    if outputflag =='score' :
        model.add(Dense(1,init='he_normal',W_regularizer=l2(weight_decay)))
    else:
        model.add(Dense(10, activation='softmax',init='he_normal',W_regularizer=l2(weight_decay)))    

    if weights_path:
        model.load_weights(weights_path)

    return model

In [4]:
## load the network for predicting aesthetics
outputflag = 'score'
path_weights = '/data/bjin/MyAesthetics/model_weights/'
weights_file = 'AVA_AestheticNet_' + outputflag + '_ICIP.h5'
model = AestheticNet(outputflag=outputflag,weight_decay=0.0005, weights_path=path_weights + weights_file)


In [37]:
#### Takeing a square crop out of the image
print('Taking the aesthetically pleasing crop')
n_crops = 500;
thre_dim = 0.5

foldername = '05-Forrest-jungle_view'
path_dataset = '/data/bjin/MyAesthetics/Hotel_images/images/'
path_results = '/data/bjin/MyAesthetics/Hotel_images/crops/'

files = [x for x in os.listdir(path_dataset+foldername + '/') if os.path.isfile(path_dataset+foldername + '/'+x)]
# files = sorted(files)
n_files = len(files)

try:
    os.mkdir(path_results+ foldername + '/')
except:
    pass

predict_filename = path_dataset + foldername + '_Scores.txt'
f = open(predict_filename,'w+')

for i in tqdm(range(n_files)):
    filename = files[i]
    if filename[0] is '.':
        continue
    
    if os.path.isfile(path_results+foldername + '/'+filename):
        continue
    
    image = cv2.imread(path_dataset+foldername + '/'+filename)
    if image is None:
        continue
    image = image.astype(np.float32, copy=False)
    imgn = np.copy(image)
    image_sz = image.shape

    ## compute the score for the original image
    imgn = cv2.resize(imgn,(224,224))
    imgn = np.transpose(imgn,[2,0,1])
    imgn = np.expand_dims(imgn,axis=0)
    score_ori = model.predict(imgn)
    score_best = 0
    image_patch_best = None
    ## loop over all crops, find the one that has higher score than the original image
    for j in range(n_crops):
        temp = np.min([image_sz[0],image_sz[1]])
        height_crop = random.randint(int(thre_dim*temp),int(temp))
        width_crop = height_crop

        start_r = random.randint(0,image_sz[0]-height_crop)
        start_c = random.randint(0,image_sz[1]-width_crop)
        end_r = start_r + height_crop
        end_c = start_c + width_crop

        image_patch = image[start_r:end_r,start_c:end_c,:]
        imgn = cv2.resize(image_patch,(224,224))
        imgn = np.transpose(imgn,[2,0,1])
        imgn = np.expand_dims(imgn,axis=0)
        score_crop = model.predict(imgn)

        if score_crop > score_best:
            score_best = score_crop
            image_patch_best = image_patch

    # save the best crop
    cv2.imwrite(path_results+foldername + '/'+filename, image_patch_best)
    
    # write down the scores
    line = filename + ' ori: ' + str(score_ori) + ' crop: ' + str(score_best)
    print >>f, line
f.close()


  0%|          | 0/331 [00:00<?, ?it/s]

Taking the aesthetically pleasing crop


100%|██████████| 331/331 [1:08:33<00:00, 13.69s/it]


In [51]:
#### Takeing a square crop out of the image
print('Taking the aesthetically pleasing crop')
n_crops = 150;
thre_dim = 0.5

foldername = 'Bathroom'
path_dataset = '/data/bjin/MyAesthetics/Hotel_images/images/'
path_results = '/data/bjin/MyAesthetics/Hotel_images/crops/'

files = [x for x in os.listdir(path_dataset+foldername + '/') if os.path.isfile(path_dataset+foldername + '/'+x)]
# files = sorted(files)
n_files = len(files)

try:
    os.mkdir(path_results+ foldername + '/')
except:
    pass

predict_filename = path_dataset + foldername + '_Scores.txt'
f = open(predict_filename,'w+')

for i in tqdm(range(n_files)):
    filename = files[i]
    if filename[0] is '.':
        continue
    
    if os.path.isfile(path_results+foldername + '/'+filename):
        continue
    
    image = cv2.imread(path_dataset+foldername + '/'+filename)
    if image is None:
        continue
    image = image.astype(np.float32, copy=False)
    imgn = np.copy(image)
    image_sz = image.shape

    ## compute the score for the original image
    imgn = cv2.resize(imgn,(224,224))
    imgn = np.transpose(imgn,[2,0,1])
    imgn = np.expand_dims(imgn,axis=0)
    score_ori = model.predict(imgn)
    
    ## find the crop with best aesthetics score
    score_best = 0
    coordinate_best = None
    width_best = None
    images_batch = np.zeros((n_crops,3,224,224))
    coordinates_batch = np.zeros((n_crops,2))

    ratio = float(224) / np.min(image_sz[0:2])
    imgn = cv2.resize(image,(0,0),fx=ratio,fy=ratio)
    image_szn = imgn.shape
    imgn = np.transpose(imgn,[2,0,1])
    for cropcounter in range(n_crops):
        start_r = random.randint(0,image_szn[0]-224)
        start_c = random.randint(0,image_szn[1]-224)
        end_r = start_r + 224
        end_c = start_c + 224

        image_crop = imgn[:,start_r:end_r,start_c:end_c]
        images_batch[cropcounter] = image_crop
        coordinates_batch[cropcounter] = [start_r, start_c]

    scores_batch = model.predict_on_batch(images_batch)[:]
    score_batch_bestidx = np.argmax(scores_batch[:])
    if scores_batch[score_batch_bestidx] > score_best:
        score_best = scores_batch[score_batch_bestidx]
        coordinate_best = (coordinates_batch[score_batch_bestidx] / ratio).astype(np.uint8)
        width_best = int(224/ratio)

    ratio = float(336) / np.min(image_sz[0:2])
    imgn = cv2.resize(image,(0,0),fx=ratio,fy=ratio)
    image_szn = imgn.shape
    imgn = np.transpose(imgn,[2,0,1])
    for cropcounter in range(n_crops):
        start_r = random.randint(0,image_szn[0]-224)
        start_c = random.randint(0,image_szn[1]-224)
        end_r = start_r + 224
        end_c = start_c + 224

        image_crop = imgn[:,start_r:end_r,start_c:end_c]
        images_batch[cropcounter] = image_crop
        coordinates_batch[cropcounter] = [start_r, start_c]

    scores_batch = model.predict_on_batch(images_batch)[:]
    score_batch_bestidx = np.argmax(scores_batch[:])
    if scores_batch[score_batch_bestidx] > score_best:
        score_best = scores_batch[score_batch_bestidx]
        coordinate_best = (coordinates_batch[score_batch_bestidx] / ratio).astype(np.uint8)
        width_best = int(224/ratio)

    ratio = float(448) / np.min(image_sz[0:2])
    imgn = cv2.resize(image,(0,0),fx=ratio,fy=ratio)
    image_szn = imgn.shape
    imgn = np.transpose(imgn,[2,0,1])
    for cropcounter in range(n_crops):
        start_r = random.randint(0,image_szn[0]-224)
        start_c = random.randint(0,image_szn[1]-224)
        end_r = start_r + 224
        end_c = start_c + 224

        image_crop = imgn[:,start_r:end_r,start_c:end_c]
        images_batch[cropcounter] = image_crop
    scores_batch = model.predict_on_batch(images_batch)[:]
    score_batch_bestidx = np.argmax(scores_batch[:])
    if scores_batch[score_batch_bestidx] > score_best:
        score_best = scores_batch[score_batch_bestidx]
        coordinate_best = (coordinates_batch[score_batch_bestidx] / ratio).astype(np.uint8)
        width_best = int(224/ratio)

    crop_best = image[coordinate_best[0]:coordinate_best[0]+width_best,coordinate_best[1]:coordinate_best[1]+width_best,:]
    # save the best crop
    cv2.imwrite(path_results+foldername + '/'+filename, crop_best)
    
    # write down the scores
    line = filename + ' ori: ' + str(score_ori) + ' crop: ' + str(score_best)
    print >>f, line
f.close()


  0%|          | 0/201 [00:00<?, ?it/s]

Taking the aesthetically pleasing crop


100%|██████████| 201/201 [17:05<00:00,  5.20s/it]


In [78]:
#### Takeing a square crop out of the image
print('Taking the aesthetically pleasing crop')
n_crops = 250;
thre_dim = 0.5

foldername = 'Bathroom'
path_dataset = '/data/bjin/MyAesthetics/Hotel_images/images/'
path_results = '/data/bjin/MyAesthetics/Hotel_images/crops/'

files = [x for x in os.listdir(path_dataset+foldername + '/') if os.path.isfile(path_dataset+foldername + '/'+x)]
# files = sorted(files)
n_files = len(files)

try:
    os.mkdir(path_results+ foldername + '/')
except:
    pass

for i in tqdm(range(n_files)):
    filename = files[i]
    if filename[0] is '.':
        continue
    
    if os.path.isfile(path_results+foldername + '/'+filename):
        continue
    
    image = cv2.imread(path_dataset+foldername + '/'+filename)
    if image is None:
        continue
    image_sz = image.shape

    ## find the crop with best aesthetics score
    score_best = 0
    coordinate_best = None
    images_batch = np.zeros((n_crops,3,224,224))
    coordinates_batch = np.zeros((n_crops,4))
#     if image_sz[0] <= 460 or image_sz[0] <= 840:
#         continue
    
    for cropcounter in range(n_crops):
        r_temp = random.randint(int(image_sz[0]/2),image_sz[0])
#         c_temp = random.randint(int(image_sz[1]/2),image_sz[1])
        c_temp = int(r_temp / 9 * 16)
        if image_sz[1]-c_temp < 1:
            continue
        start_r = random.randint(0,image_sz[0]-r_temp)
        start_c = random.randint(0,image_sz[1]-c_temp)
        end_r = start_r + r_temp
        end_c = start_c + c_temp
        image_crop = image[start_r:end_r,start_c:end_c,:]
        image_crop = cv2.resize(image_crop,(224,224))
        image_crop = np.transpose(image_crop,[2,0,1])
        
        images_batch[cropcounter] = image_crop
        coordinates_batch[cropcounter] = [start_r, start_c,end_r,end_c]
    
    scores_batch = model.predict_on_batch(images_batch)[:]
    score_batch_bestidx = np.argmax(scores_batch)
    [start_r, start_c,end_r,end_c] = coordinates_batch[score_batch_bestidx]
    
    crop_best = image[start_r:end_r,start_c:end_c,:]
    
    # save the best crop
    cv2.imwrite(path_results+foldername + '/'+filename, crop_best)
    

  0%|          | 0/201 [00:00<?, ?it/s]

Taking the aesthetically pleasing crop


/home/bjin/anaconda2/envs/keras1/lib/python2.7/site-packages/ipykernel/__main__.py:61: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
100%|██████████| 201/201 [09:31<00:00,  2.95s/it]


In [73]:
print image_sz, filename

(1024, 1280, 3) Beach-Resort-HD-Picture.jpeg


In [ ]:
path_images = '/data/bjin/MyAesthetics/Hotel_images/deliver/v1/crops/'

files = [x for x in os.listdir(path_images) if os.path.isfile(path_images+x)]
# files = sorted(files)
n_files = len(files)

for i in range(n_files):
    filename = files[i]
    if filename[0] is '.':
        continue
    
    image = cv2.imread(path_images+filename)
    image = image.astype(np.float32, copy=False)
    imgn = np.copy(image)
    image_sz = image.shape

    ## compute the score for the original image
    imgn = cv2.resize(imgn,(224,224))
    imgn = np.transpose(imgn,[2,0,1])
    imgn = np.expand_dims(imgn,axis=0)
    score_ori = model.predict(imgn)
    print filename, score_ori


In [ ]:
## load the image 
path_AVA = '/data/bjin/AVA/'
metadata = loadmat(path_AVA + 'AVA_metadata.mat')
files = metadata['img_names'][0,:]
GTscores = metadata['scores']
n_files = len(files)
print n_files
idx = np.random.permutation(n_files)
files = files[idx]
GTscores = GTscores[:,idx]

path_results = '/home/bjin/SampleImages/'

files = metadata['img_names'][0,:]
GTscores = metadata['scores']
mystd = np.std(GTscores[1:,:],axis=0)
idx = np.argsort(mystd)
files = files[idx]
GTscores = GTscores[:,idx]

for filename, GTscore in zip(files,GTscores.transpose([1,0])):
#     if GTscore[0] > 7 or GTscore[0] < 5:
#         continue
    
    img = cv2.imread(path_AVA + 'img/' + str(filename[0]))
    imgn = np.copy(img)
    imgn = cv2.resize(imgn,(224,224))
    imgn = np.transpose(imgn,[2,0,1])
    imgn = np.expand_dims(imgn,axis=0)
    score = model.predict(imgn)
    
    fig = plt.figure(figsize=(20, 4))
    ax1 = fig.add_subplot(121)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    ax1.set_title('ori')
    ax1.axis('off')
    
    ax2 = fig.add_subplot(122)
    plt.bar(range(10),GTscore[1:]/np.sum(GTscore[1:]),width=0.9,align='center')
    plt.xticks(range(1,11),('1', '2', '3', '4', '5', '6','7','8','9','10'))
    plt.xlim(.5, 10.5)
    ax2.set_title('hist')
    plt.show()
    print score, GTscore
    
    flag = raw_input('press "s" to save: ')
    if flag == 's':
        extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(path_results + str(filename[0]), bbox_inches=extent)

        extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(path_results + 'hist_' + str(filename[0]), bbox_inches=extent.expanded(1.1, 1.2))
    
    plt.clf()
    
